# 文本分类实例 - 中文评价情绪预测（Text Classification Example - Chinese Evaluation Emotion Prediction）

### 数据集: 某外卖平台收集的用户评价，正向 4000 条，负向 约 8000 条

## 字段说明

| 字段 | 说明 |
| ---- | ---- |
| label | 1 表示正向评论，0 表示负向评论 |
| review | 评论内容 |

安装 jieba 和 pandas

     pip install pandas jieba sklearn -i https://pypi.doubanio.com/simple

In [1]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import jieba
import matplotlib.pyplot as plt
%matplotlib inline

## 数据读取和观察

In [2]:
# 读取数据
data = pd.read_csv('./dataset/waimai_10k.csv')

In [3]:
# 查看数据
data.head()

,label,review
0,1,很快，好吃，味道足，量大
1,1,没有送水没有送水没有送水
2,1,非常快，态度好。
3,1,方便，快捷，味道可口，快递给力
4,1,菜味道很棒！送餐很及时！


In [4]:
# 查看数据集信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11987 entries, 0 to 11986
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   11987 non-null  int64 
 1   review  11987 non-null  object
dtypes: int64(1), object(1)
memory usage: 187.4+ KB


In [5]:
# 查看数据集标签分布
data.label.value_counts()

0    7987
1    4000
Name: label, dtype: int64

## 对文本数据进行分词

In [6]:
# 文本处理函数，去除标点符号，分词
def pre_text(text):
    text = text.replace('！', '').replace('，', '').replace('。', '')
    return jieba.lcut(text)

In [7]:
# 对文本进行预处理、分词
data['review'] = data.review.apply(pre_text)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LIUBAI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.669 seconds.
Prefix dict has been built successfully.


In [8]:
# 查看分词后的数据
data.review

0                                      [很快, 好吃, 味道, 足量, 大]
1                                 [没有, 送水, 没有, 送水, 没有, 送水]
2                                           [非常, 快, 态度, 好]
3                                 [方便快捷, 味道, 可口, 快, 递给, 力]
4                                   [菜, 味道, 很棒, 送餐, 很, 及时]
                               ...                        
11982                   [以前, 几乎, 天天, 吃, 现在, 调料, 什么, 都, 不放]
11983    [昨天, 订, 凉皮, 两份, 什么, 调料, 都, 没有, 放, 就, 放, 了, 点, ...
11984                                  [凉皮, 太辣, ,, 吃不下, 都]
11985                                [本来, 迟到, 了, 还, 自己, 点]
11986    [肉夹馍, 不错, 羊肉, 泡馍, 酱肉, 包, 很, 一般, 凉面, 没, 想象, 中, ...
Name: review, Length: 11987, dtype: object

## 创建词表（vocab）

In [9]:
# 将分词后的所有词添加到一个array中后，统计词频
word_count = pd.value_counts(np.concatenate(data.review.values))
# 过滤掉词频小于3的词
word_count = word_count[word_count > 2]

# 查看词频统计结果
word_count

了       9397
的       7836
,       4212
很       2257
都       2192
        ... 
精神         3
停不下来       3
过敏         3
要不然        3
筋道         3
Length: 3870, dtype: int64

In [10]:
# 获取词表长度
max_word = len(word_count) + 1
max_word

3871

In [11]:
# 将词频统计结果转换为列表并编码
word_list = list(word_count.index)
# 查看编码后的词列表
word_list.index('好吃')

12

In [12]:
# 将词列表转换为字典，使用位置进行编码
word_index =  dict((word, word_list.index(word) + 1) for word in word_list)
word_index

{'了': 1,
 '的': 2,
 ',': 3,
 '很': 4,
 '都': 5,
 '是': 6,
 '我': 7,
 '也': 8,
 '不': 9,
 '还': 10,
 '好': 11,
 '味道': 12,
 '好吃': 13,
 '送餐': 14,
 '吃': 15,
 '送': 16,
 '就': 17,
 '不错': 18,
 '小时': 19,
 '给': 20,
 '没有': 21,
 '没': 22,
 '？': 23,
 '点': 24,
 '送到': 25,
 '说': 26,
 '…': 27,
 '速度': 28,
 '就是': 29,
 '等': 30,
 '才': 31,
 '外卖': 32,
 '到': 33,
 '太': 34,
 '快': 35,
 '难吃': 36,
 '在': 37,
 '一个': 38,
 '啊': 39,
 '菜': 40,
 '一般': 41,
 '送来': 42,
 '太慢': 43,
 '非常': 44,
 '还是': 45,
 '时间': 46,
 '饭': 47,
 '凉': 48,
 '多': 49,
 '和': 50,
 '有': 51,
 '有点': 52,
 '吧': 53,
 '很快': 54,
 '可以': 55,
 '个': 56,
 '慢': 57,
 '配送': 58,
 '百度': 59,
 '～': 60,
 '特别': 61,
 '但是': 62,
 '两个': 63,
 '态度': 64,
 '粥': 65,
 '这': 66,
 '要': 67,
 '而且': 68,
 '不是': 69,
 '什么': 70,
 '肉': 71,
 '少': 72,
 '打电话': 73,
 '电话': 74,
 '饼': 75,
 '让': 76,
 '人': 77,
 '服务': 78,
 '再': 79,
 '分钟': 80,
 '你': 81,
 '吗': 82,
 '又': 83,
 '小哥': 84,
 '怎么': 85,
 '能': 86,
 '以后': 87,
 '差': 88,
 '快递': 89,
 '东西': 90,
 '结果': 91,
 '米饭': 92,
 '量': 93,
 '这么': 94,
 '感觉': 95,
 '~': 96,
 '里':

## 文本数据预处理

In [13]:
# 将文本转换为编码后的数字列表
text = data.review.apply(lambda x: [word_index.get(word, 0) for word in x])
text

0                                     [54, 13, 12, 0, 113]
1                           [21, 3168, 21, 3168, 21, 3168]
2                                         [44, 35, 64, 11]
3                          [3112, 12, 1388, 35, 2634, 517]
4                                [40, 12, 392, 14, 4, 290]
                               ...                        
11982          [229, 560, 1455, 15, 287, 960, 70, 5, 2151]
11983    [618, 110, 379, 338, 70, 960, 5, 21, 198, 17, ...
11984                               [379, 955, 3, 2642, 5]
11985                           [381, 376, 1, 10, 154, 24]
11986    [565, 18, 887, 1678, 3146, 634, 4, 41, 767, 22...
Name: review, Length: 11987, dtype: object

In [14]:
# 获取文本的最大长度
maxlen = max(len(x) for x in text)
print("Max length of text: ", maxlen)

Max length of text:  279


In [15]:
# 设置文本长度为20
text_len = 20
# 对文本进行填充或截断，使其长度为20
pad_text = [l + (text_len-len(l))*[0] if len(l)<=text_len else l[ :text_len] for l in text]
# 转换为nparray
pad_text = np.array(pad_text)
# 查看填充后的文本形状
print("Shape of padded text: ", pad_text.shape)

Shape of padded text:  (11987, 20)


## 标签数据预处理

In [16]:
# 获取标签列表
labels = data.label.values
# 查看标签列表形状
print("Shape of labels: ", labels.shape)

Shape of labels:  (11987,)


## 划分数据集

In [17]:
from sklearn.model_selection import train_test_split
# 划分数据集，使用默认的划分比例7.5：2.5
x_train, x_test, y_train, y_test = train_test_split(pad_text, labels)
x_train.shape, x_test.shape, y_train.shape

((8990, 20), (2997, 20), (8990,))